In [91]:
# load dependencies
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import json
import pickle

# Agenda

## Focus of today
* Recap of the essentials of web scraping.
    * Navigating html, reliable connection, parsing, storing, logging.

* Showing you some tricks of the trade
* Hands on with APIs

... 
* Pointing to the skills you need to master.

## Motivation
<a href="https://imgflip.com/i/24m2p1"><img src="https://i.imgflip.com/24m2p1.jpg" title="made at imgflip.com"/></a>


## Main take-away
* Central problem you are facing is *ensuring* and *demonstrating* **data quality** by:
    * Logging, 
        * the scraping process
        * the parse
    * ensuring reproducibility (storing raw data)        
    * inspections (random selections of your parse)
    * exploratory data analysis.



# Quick setup - connecting and legal notice

## Ethics / Legal Issues
* If a regular user can’t access it, we shouldn’t try to get it (That is considered hacking)https://www.dr.dk/nyheder/penge/gjorde-opmaerksom-paa-cpr-hul-nu-bliver-han-politianmeldt-hacking. 
* Don't hit it to fast: Essentially a DENIAL OF SERVICE attack (DOS). [Again considered hacking](https://www.dr.dk/nyheder/indland/folketingets-hjemmeside-ramt-af-hacker-angreb). 
* Add headers stating your name and email with your requests to ensure transparency. 
* Be careful with copyrighted material.
* Fair use (don't take everything)
* If monetizing on the data, be careful not to be in direct competition with whom you are taking the data from.
* Who are you collecting data about?
    * Political groups, minorities or organizations, or all at the same time?
     

## Connecting

In [2]:
# Transparent scraping
import requests
#response = requests.get('https://www.google.com')
session = requests.session()
session.headers['email'] = 'youremail' 
session.headers['name'] = 'name'
session.headers

{'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'email': 'youremail', 'name': 'name'}

A quick tip is that you can change the user agent to a cellphone to obtain more simple formatting of the html. 

In [3]:
# Control the pace of your calls
import time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # change to design more clever schemes for handling the rate of calls.
# Reliable requests
def request(url,iterations=10,exceptions=(Exception)):
    """This module ensures that your script does not crash from connection errors.
        iterations : Define number of iterations before giving up. 
        exceptions: Define which exceptions you accept, default is all. 
    """
    for iteration in range(iterations):
        try:
            # add ratelimit function call here
            ratelimit() # !!
            response = session.get(url)
            return response # if succesful it will end the iterations here
        except exceptions as e: #  find exceptions in the request library requests.exceptions
            print(e) # print or log the exception message.
    return None # your code will purposely crash if you don't create a check function later.

In [ ]:
# Interactive browsing. 
# Need to download geckodrivers (https://github.com/mozilla/geckodriver/releases) and install selenium first
from selenium import webdriver
path2gecko = '/Users/axelengbergpallesen/Downloads/geckodriver' # define path to your geckodriver
browser = webdriver.Firefox(executable_path=path2gecko)
browser.get('https://www.facebook.com')

# Essentials of webscraping

## Essentials (1): Reading html
* Using the Inspector
    * locating paths to elements: e.g css selectors
    * structure of the html-tree
    * uncover hidden information e.g. extra meta data.
    * Find direct links to the underlying database. Visit (https://www.jobnet.dk)

In [ ]:
url = ''
request(url)

## Essentials(2): Navigation
- http hacking: Visit https://www.jobindex.dk
- crawling and following links
- interactions 
    * send keys: e.g. login 
    * clicks
    * scroll-down (https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)

In [ ]:
# hardcoded interaction
css_sel = ''
element = browser.find_element_by_css_selector(css_sel)
element.click()

## Essentials(3): Parsing


In [7]:
import re # generic pattern search
from bs4 import BeautifulSoup # navigating the html tree
'Scraping,2,is,3,easy,6,?'.split(',')[::2] # good old split function

['Scraping', 'is', 'easy', '?']

![](http://www.tankonyvtar.hu/hu/tartalom/tamop412A/2011_0015_kliens_oldali_technologiak_EN/files/4018.jpg)


In [10]:
url = 'https://it.jobindex.dk/jobsoegning?maxdate=20180214&mindate=20171101&jobage=archive'
response = request(url)
soup = BeautifulSoup(response.text,'lxml')

In [12]:
# lets find the section with the results.
results = soup.findAll('div',{'class':'results  component--default'})
results


[]

** shit! ** it is dynamically generated. 

Now we have the possibility of using selenium or inpecting the network acitivty / following the html build instructions.

In [22]:
url = 'https://it.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20171101&maxdate=20180214&page=0'
response = request(url)
if response.ok:
    
    print('response is OK')
    soup = BeautifulSoup(response.json()['result_list_box_html'],'lxml')
    

response is OK


In [31]:
results = soup.find('div',{'class':'results component--default'})
for child in results.findAll('div'):
    break
    # getting links.
    relative_link = child.a['data-click']

    # getting headlines.
    headline = child.b.text
    # company
    company = child.p.a.b.text

## Essentials(4): Storing, and formatting

In [ ]:
open(filename,'w')
#formats
#raw,pickle,json, csv
f.write
json.dump(), json.dumps
csv.writer
pd.to_csv,pd.to_pickle,pd.to_feather 
# for arbitrary python objects
pickle.dump
# Databases.

In [ ]:
! mkdir jobindex_raw

In [ ]:
# paging the first 5
for page_n in range(5):
    url = 'https://it.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20171101&maxdate=20180214&page=%d'%page_n
    response = request(url)
    # store it.
    if response.ok:
        path = 'jobindex/jobindex_page_%d'%page_n
        f = open(path,'w')
        f.write(response.text)
        f.close()
    

## Essentials (5): Logging and Quality Controls

In [5]:
response = request('https://www.google.com')

In [ ]:
# setting up the logfile
! mkdir logs
log_id = 0 
logfile = open('logs/test_log','w') 
log_header = ['log_id','status_code','timestamp','length','output_file']  # 
logfile.write(','.join(log_header)) # comment this out after the first write
logfile.close() 


In [29]:
logfile = open('logs/test_log','a') 
import time
def log(response,output_path,logfile=logfile):
    "A simple example of a logging function"
    global log_id
    # give it an id
    log_id += 1
    status_code = response.status_code
    timestamp = time.time()
    length = len(response.text)
    # Header 'log_id','status_code','timestamp','length','output_file'
    logfile.write('%d,%d,%r,%d,%s'%(log_id,status_code,timestamp,length,output_path))
    # add a write function here
    
! mkdir test_scrape
log(response,'test_scrape/google.com')

13236

## Quality of the parse
* Number of missing values accross different Variables, and accross different sites or subsections of a site.
* If temporal data: quality accross time.
* Profile missing values
    * Might be related to time of day, rate of calls, length of the response, or other more complex factors logged.
* Do many random samples of the parse and checks of the final dataframe or dataset.
    * E.g. do random sample in pandas like this: df.sample(frac=0.1).head()



# Scraping the Cryptomarkets to investigate rate of success, bots and scams.

## Exercise (1)
Visit following page: https://coinmarketcap.com/

The page allows you to investigate the ***"big"*** players of the cryptomarket. 

** Navigation **
- Find a way to navigate the page and extract links to the blockchain companies.



In [ ]:
# Solution here

## Exercise (2)
Now visit one of the links and see what data you can get.

**Backdoors**
- Look at the network monitor and locate the link to the file with the historical trading data (i.e. the graph displayed). 
- See how the link is constructed, and make a loop that collects and stores the first 5 companies. 


In [ ]:
# Solution here

## Exercise (3)
** HTML parsing **

Now we need to visit the pages and collect data stored in the html.

Use the links collected before and store the first 5 raw html responses

* Design a parser that collects the Twitter handle of each company - from the Social Media Feed option (+whatever meta data you like) - and apply it to the raw html.
    - Note not all companies has a twitter handle.


In [ ]:
# Solution here

# APIs
For fast, efficient and ***reliable*** data collection.

Only catch is that they control the amounts, and which endpoints you can collect 

also they **change**.
    - e.g. facebook cancelled 
        - querying friendship relations (without having using signing up to your app), 
        - group activity without admin rights, 
        - and most recently the ability to trace public activity (likes and comments) without admin rights.
    - twitter (and more recently facebook) will not let you collect all historic activity --> streaming data.

Begins with reading the docs... 
- getting authentification - creating apps, getting and renewing tokens - 
- building queries.
- ratelimiting and pagination.

Often comes in the Json format. --> nested dictionaries and lists.

Example: Explore the facebook api here: https://developers.facebook.com/tools/explorer/

## APIS: Collect data from Twitter

In [151]:
# XXX: Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials that you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation

CONSUMER_KEY=""
CONSUMER_SECRET=""
OAUTH_TOKEN=""
OAUTH_TOKEN_SECRET=""

In [ ]:
#pickle.dump([CONSUMER_KEY,CONSUMER_SECRET,OAUTH_TOKEN,OAUTH_TOKEN_SECRET],open('twitter_credentials.pkl','wb'))
CONSUMER_KEY,CONSUMER_SECRET,OAUTH_TOKEN,OAUTH_TOKEN_SECRET = pickle.load(open('twitter_credentials.pkl','rb'))

In [193]:
# answer from https://stackoverflow.com/questions/33308634/how-to-perform-oauth-when-doing-twitter-scrapping-with-python-requests
from requests_oauthlib import OAuth1
#url = 'https://api.twitter.com/1.1/account/verify_credentials.json' # verify that your authorization works.
#url = 'https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=singularity_net&count=5000' # get timeline

# Description of the call https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-list
url = 'https://api.twitter.com/1.1/followers/list.json?cursor=-1&screen_name=%s&count=5000'%twitter_name

auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
response = requests.get(url, auth=auth).json()

In [196]:
# define container to store the responses.
responses = []

In [198]:
cursor = -1 # start with -1
url = 'https://api.twitter.com/1.1/followers/list.json?cursor=%d&screen_name=singularity_net&count=5000' %cursor
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
response = requests.get(url, auth=auth).json()
responses.append(response)
count = 0
while cursor!=0:
    time.sleep(1)
    cursor = response['next_cursor']
    url = 'https://api.twitter.com/1.1/followers/list.json?cursor=%d&screen_name=singularity_net&count=5000'%cursor
    response = requests.get(url, auth=auth).json()
    # dump response
    responses.append(response)
    count+=1
    if count%10==0:
        print(count,end=' ')
#response

10 

KeyError: 'next_cursor'

In [199]:
response

{'errors': [{'code': 88, 'message': 'Rate limit exceeded'}]}

Packages: python-twitter and tweepy.

# Exercise(4)
- Collect the link to each of the crypto-companies' github page (Source Code) and get data from the api.
    * E.g. Get the contributors of the repository. - Count the number of unique contributors.
    * Get events. 
    * Get forks.
    

## GITHUB API
Check the docs:
https://developer.github.com/v3/

### Practice with some calls###
The github api is nice to work with because it provides links to further calls that can be made.
* This will list basic queries: https://api.github.com/
* Play around with it: e.g. https://api.github.com/repos/singnet/singnet/contributors 

### Read about paging
here: https://developer.github.com/v3/guides/traversing-with-pagination/
### Rate limits
For practice you can make calls without authorization. However if you need large amounts of data you need an authorization token.
This means the standard procedure: https://developer.github.com/apps/building-oauth-apps/authorization-options-for-oauth-apps/
    * Create an app here (under developers settings): https://github.com/settings/, collect client_id and client_secret for the app.
    * Let a user (yourself) Authorize your app. Using the specified procedure described https://github.com/login/oauth/authorize?client_id=(indsæt_client_id).
    * Get the code returned (look at the url in the top of the browser).
    * Use the code, client_id, client_secret to request an access_token: 'https://github.com/login/oauth/access_token?code=%s&client_id=%s&client_secret=%s'%(code,client_id,client_secret)




## ressource a guide to authentication

In [214]:
# look at the guide for making authorized calls:
# https://developer.github.com/apps/building-oauth-apps/authorization-options-for-oauth-apps/
client_id = ''#found in the settings of your newly generated test app
client_secret = '' # also found in the settings of your app
code = ''# found using the call  https://github.com/login/oauth/authorize?client_id=%s'%client_id

In [221]:
#pickle.dump([client_id,client_secret,code],open('github_credentials.pkl','wb'))
client_id,client_secret,code = pickle.load(open('github_credentials.pkl','rb'))

In [234]:
# as described in the docs https://developer.github.com/apps/building-oauth-apps/authorization-options-for-oauth-apps/
url = 'https://github.com/login/oauth/access_token?code=%s&client_id=%s&client_secret=%s'%(code,client_id,client_secret)
response = request(url)
token = response.text.split('access_token=')[0].split('&')[0]

In [92]:
#pickle.dump(token,open('gittoken','wb'))
token = pickle.load(open('gittoken','rb'))

In [ ]:
# Solutions here

In [246]:
# list possible calls: https://api.github.com/

# check your ratelimit: 'https://api.github.com/rate_limit?access_token=%s'%token
# interesting calls
# https://api.github.com/repos/singnet/singnet/contributors # contributors to a project, create a network of co-contributors.
# 'https://api.github.com/repos/singnet/singnet/forks'
#'https://api.github.com/repos/singnet/singnet/events' # get all events (watch and fork, pull, push and commit events), are they coordinated?
## are they from bots. e.g. dmonroe3 who follows 5 blockchain things.
## Also includes push events and messages.
# 'https://api.github.com/users/dmonroe3/followers # get the followers of a user
# https://api.github.com/users/thinxtank # get basic info about a user.


### paging

In [245]:
url ='https://api.github.com/repos/singnet/singnet/forks'
response = request(url)
response.headers['Link'] # Here you can read how many pages to page. read here: https://developer.github.com/v3/guides/traversing-with-pagination/

'<https://api.github.com/repositories/100654282/forks?page=2>; rel="next", <https://api.github.com/repositories/100654282/forks?page=3>; rel="last"'

# Perspectives
## Regex
For cleaning, further extraction of information from html and text, you need to learn Regular expressions --> More to come in the future sessions. 

    - For now. Go play around at https://www.regexr.com.

or do the following exercises:
- Locate the link to the website of each of the blockchain companies. 
- Try to find the number of doctors displayed on the website of each project (frontpage or crawl the whole map, while your at it see if you can do some other stats about the team.). Match the name after the Dr. prefix.
- Find the numbers thrown around on each website: $9234 million / trillion / thousands. 
    - Find the previous word before the number.

# Perspectives (2)
## Common scenarios
Tabular data from html.
    - Beautifulsoup: Google is your friend (e.g. https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table)
    - Pandas: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html

Interactions(clicking, scrolling, logins etc):
    * Use selenium.
        - Select element (using css-selector path or search as in beautifulsoup)
            - element.click()
            - element.send_keys()
        - scrolling (google "scroll to the bottom selenium python")

# Perspectives(3)
### Reliability
EDA